In [2]:
import numpy as np
import random

# Euclidean distance function between two points (customers)
def euclidean_distance(a, b):
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

# Fitness function (total distance traveled by all vehicles)
def calculate_total_distance(routes, customers, vehicle_capacity, num_vehicles):
    total_distance = 0
    for v in range(num_vehicles):
        route = routes[v]
        route_distance = 0
        # Ensure the route includes the depot at the start and end
        full_route = [0] + route + [0]
        for i in range(len(full_route) - 1):
            route_distance += euclidean_distance(customers[full_route[i]], customers[full_route[i + 1]])
        total_distance += route_distance
    return total_distance

# PSO Algorithm
def pso_vrp(customers, num_vehicles, max_iter=100, N=30, w=0.5, c1=1.5, c2=1.5):
    # Number of particles
    num_customers = len(customers) - 1  # Exclude depot (customer 0)

    # Initialize the swarm of particles
    swarm = []
    best_positions = []
    best_fitness = []
    global_best_position = None
    global_best_fitness = float('inf')

    # Initialize particles
    for _ in range(N):
        # Randomly shuffle customer indices (excluding depot) for each vehicle
        particles = [random.sample(range(1, num_customers + 1), num_customers) for _ in range(num_vehicles)]
        swarm.append(particles)

        # Calculate the fitness of each particle (total distance of all routes)
        # We use num_customers as a placeholder for vehicle_capacity since it's not used in calculate_total_distance
        fitness = calculate_total_distance(particles, customers, num_customers, num_vehicles)
        best_positions.append(particles)
        best_fitness.append(fitness)

        # Update global best
        if fitness < global_best_fitness:
            global_best_fitness = fitness
            global_best_position = particles

    # Particle velocity initialization (random vectors for each vehicle's route)
    velocities = [[np.random.rand(num_customers) * 2 - 1 for _ in range(num_vehicles)] for _ in range(N)] # Random velocities as lists of arrays

    # PSO main loop
    for iter in range(max_iter):
        for i in range(N):
            # Update particle velocity and position (treating routes as lists)
            r1, r2 = np.random.rand(), np.random.rand()  # Random factors
            new_velocities = []
            new_positions = []

            for v in range(num_vehicles):
                # Convert lists to arrays for calculations, then back to lists for positions
                current_pos_array = np.array(swarm[i][v])
                best_pos_array = np.array(best_positions[i][v])
                global_best_pos_array = np.array(global_best_position[v])

                # Ensure arrays have the same length before operations
                min_len = min(len(current_pos_array), len(best_pos_array), len(global_best_pos_array))
                current_pos_array = current_pos_array[:min_len]
                best_pos_array = best_pos_array[:min_len]
                global_best_pos_array = global_best_pos_array[:min_len]
                velocities[i][v] = velocities[i][v][:min_len]


                new_velocity = (w * velocities[i][v] +
                                c1 * r1 * (best_pos_array - current_pos_array) +
                                c2 * r2 * (global_best_pos_array - current_pos_array))
                new_velocities.append(new_velocity)

                # Update position by adding velocity, ensuring integer customer indices
                # Clamp values to be within valid customer indices range (1 to num_customers)
                new_position_array = (current_pos_array + new_velocity).astype(int)
                new_position_array = np.clip(new_position_array, 1, num_customers)

                # Convert back to a list of unique customer indices, preserving order as much as possible
                # Create a unique list while trying to maintain the order from the new position
                seen = set()
                new_route = []
                for customer_idx in new_position_array:
                    if customer_idx not in seen:
                        new_route.append(int(customer_idx)) # Ensure int type
                        seen.add(customer_idx)

                # If the route is shorter than the number of customers, add missing unique customers
                missing_customers = set(range(1, num_customers + 1)) - set(new_route)
                new_route.extend(list(missing_customers))
                random.shuffle(new_route) # Shuffle added missing customers

                new_positions.append(new_route)

            swarm[i] = new_positions
            velocities[i] = new_velocities # Update velocities with the potentially truncated versions

            # Calculate fitness (total distance)
            # We use num_customers as a placeholder for vehicle_capacity since it's not used in calculate_total_distance
            fitness = calculate_total_distance(swarm[i], customers, num_customers, num_vehicles)

            # Update the personal best
            if fitness < best_fitness[i]:
                best_fitness[i] = fitness
                best_positions[i] = swarm[i]

            # Update the global best
            if fitness < global_best_fitness:
                global_best_fitness = fitness
                global_best_position = swarm[i]

        print(f"Iteration {iter + 1}: Best Fitness = {global_best_fitness}")

    return global_best_position, global_best_fitness

# Example usage of PSO for VRP
if __name__ == "__main__":
    # Example customers (depot at index 0, followed by customer locations)
    # Customers are represented as (x, y) coordinates
    customers = [(0, 0), (2, 3), (3, 4), (6, 1), (8, 3)]
    num_vehicles = 2  # Number of delivery trucks

    # Running the PSO for VRP
    best_routes, best_fitness = pso_vrp(customers, num_vehicles)

    print("\nBest Routes:")
    for v in range(num_vehicles):
        print(f"Vehicle {v+1}: {best_routes[v]}")
    print(f"Total Distance: {best_fitness}")

Iteration 1: Best Fitness = 39.664810401494364
Iteration 2: Best Fitness = 39.664810401494364
Iteration 3: Best Fitness = 39.664810401494364
Iteration 4: Best Fitness = 39.664810401494364
Iteration 5: Best Fitness = 39.664810401494364
Iteration 6: Best Fitness = 39.664810401494364
Iteration 7: Best Fitness = 39.664810401494364
Iteration 8: Best Fitness = 39.664810401494364
Iteration 9: Best Fitness = 39.664810401494364
Iteration 10: Best Fitness = 38.05994801294856
Iteration 11: Best Fitness = 38.05994801294856
Iteration 12: Best Fitness = 38.05994801294856
Iteration 13: Best Fitness = 38.05994801294856
Iteration 14: Best Fitness = 38.05994801294856
Iteration 15: Best Fitness = 38.05994801294856
Iteration 16: Best Fitness = 38.05994801294856
Iteration 17: Best Fitness = 38.05994801294856
Iteration 18: Best Fitness = 38.05994801294856
Iteration 19: Best Fitness = 38.05994801294856
Iteration 20: Best Fitness = 38.05994801294856
Iteration 21: Best Fitness = 38.05994801294856
Iteration 22: